In [1]:
#some notes
#1. Doesn't look like a player receive a curse is treated as an action 
#2. Should I add round end markers?
#3. Does normalising the ngram probabilities with the number of events in the trace make sense?

In [1]:
import pandas as pd
import pdb
import re
import nltk
from itertools import product
from nltk import ngrams
from nltk.probability import FreqDist
nltk.download('punkt')
from collections import Counter

#define number of NGrams to use
N = 2

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anthonyowen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data  = pd.read_csv("data/ActionsReduced_BMWG_vs_DW_GPM100.csv")
data = data[['GameID', 'Player', 'Round','Turn','ActionDescription']]
print(data)
print(data.columns)

       GameID  Player  Round  Turn              ActionDescription
0           2       0      0     0              End Current Phase
1           2       0      0     0              End Current Phase
2           2       1      0     1              End Current Phase
3           2       1      0     1    BuyCard: SILVER by player 1
4           2       0      1     0              End Current Phase
...       ...     ...    ...   ...                            ...
17005     201       0     18     0    BuyCard: ESTATE by player 0
17006     201       1     18     1               WITCH : Player 1
17007     201       1     18     1  BuyCard: PROVINCE by player 1
17008     201       0     19     0              End Current Phase
17009     201       0     19     0  BuyCard: PROVINCE by player 0

[17010 rows x 5 columns]
Index(['GameID', 'Player', 'Round', 'Turn', 'ActionDescription'], dtype='object')


In [4]:
#kingdom card types
card_types_SD = ['ARTISAN', 'BANDIT', 'BUREAUCRAT', 'CHAPEL', 'FESTIVAL', 'GARDENS', 'SENTRY', 'THRONE_ROOM', 'WITCH',
                 'WORKSHOP', 'CURSE', 'PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']
card_types_SD_no_curse = ['ARTISAN', 'BANDIT', 'BUREAUCRAT', 'CHAPEL', 'FESTIVAL', 'GARDENS', 'SENTRY', 'THRONE_ROOM', 'WITCH',
                 'WORKSHOP','PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']
card_types = card_types_SD_no_curse

In [5]:
#first format action decription
def format_action(action, cardtypes):
    #there are ... types of actions we need to format
    # 1. 'End Current Phase' goes to ECP
    # 2. 'BuyCard: CARDX by player Y' goes to BUYCARDX_Y
    # 3. 'CARDX : Player Y' goes to PLAYCARDX_Y
    
    #to identify these we use regular expressions
    pattern1 = re.compile(r'End Current Phase')
    pattern2 = re.compile(r'BuyCard: (' + '|'.join(cardtypes) + r') by player (0|1)')
    pattern3 = re.compile(r'(' + '|'.join(cardtypes) + r') : Player (0|1)')
   
    match1 = pattern1.match(action)
    match2 = None
    match3 = None
    if match1 == None:
        match2 = pattern2.match(action)
    if match2 == None:
        match3 =  pattern3.match(action)
        
    if match1 != None:
        formatted_action = 'ECF'
    elif match2 != None:
        matched_card = match2.group(1)
        player = match2.group(2)
        formatted_action = 'BUY' + matched_card #+ '_' + str(player)
    elif match3 != None:
        matched_card = match3.group(1)
        player = match3.group(2)
        formatted_action = 'PLAY' + matched_card #+ '_' + str(player)
    else:
        raise Exception("Can't match action description")
        
    return formatted_action

#print(format_action('GOLD : Player 1', card_types_SD))
#print(format_action('BuyCard: PROVINCE by player 1', card_types))
#print(format_action('End Current Phase', card_types_SD))

In [7]:
#process input file
data['ProcAction'] = data.apply(lambda row: format_action(row['ActionDescription'], card_types), axis = 1)
print(data)

       GameID  Player  Round  Turn              ActionDescription   ProcAction
0           2       0      0     0              End Current Phase          ECF
1           2       0      0     0              End Current Phase          ECF
2           2       1      0     1              End Current Phase          ECF
3           2       1      0     1    BuyCard: SILVER by player 1    BUYSILVER
4           2       0      1     0              End Current Phase          ECF
...       ...     ...    ...   ...                            ...          ...
17005     201       0     18     0    BuyCard: ESTATE by player 0    BUYESTATE
17006     201       1     18     1               WITCH : Player 1    PLAYWITCH
17007     201       1     18     1  BuyCard: PROVINCE by player 1  BUYPROVINCE
17008     201       0     19     0              End Current Phase          ECF
17009     201       0     19     0  BuyCard: PROVINCE by player 0  BUYPROVINCE

[17010 rows x 6 columns]


In [13]:
#create list of all possible actions - note we are ignoring responses to opponents cards (for SD this means just 
#ignoring receiving curse cards

buy_actions = ['BUY' + str(card) for card in card_types]
play_actions = ['PLAY' + str(card) for card in card_types]
all_actions_list = ['ECF'] + buy_actions + play_actions
print("Action list length: " + str(len(all_actions_list)))
#print(all_actions_list)

#create list of all possible N-grams
all_ngrams_list = list(product(all_actions_list, repeat=N))
#print(all_ngrams_list)
print("N-gram list length: " + str(len(all_ngrams_list)))

Action list length: 33
N-gram list length: 1089


In [19]:
#convert input data so that each row contains gameID, player and then a list of ngrams 
#corresponding to the trace
traces = data.groupby(['GameID', 'Player'])['ProcAction'].agg(lambda x: ' '.join(x)).reset_index()
traces['NGrams'] = traces.apply(lambda row: list(ngrams(nltk.word_tokenize(row['ProcAction']),N)), axis = 1)
print(traces)

     GameID  Player                                         ProcAction  \
0         2       0  ECF ECF ECF BUYWITCH ECF BUYSILVER PLAYWITCH B...   
1         2       1  ECF BUYSILVER ECF BUYSILVER ECF BUYSILVER ECF ...   
2         3       0  ECF BUYSILVER ECF BUYSILVER ECF BUYSILVER ECF ...   
3         3       1  ECF ECF ECF BUYSILVER ECF BUYSILVER ECF BUYSIL...   
4         4       0  ECF BUYSILVER ECF BUYSILVER ECF BUYWITCH ECF B...   
..      ...     ...                                                ...   
395     199       1  ECF BUYSILVER ECF BUYSILVER ECF BUYWITCH ECF B...   
396     200       0  ECF BUYSILVER ECF BUYSILVER ECF BUYSILVER ECF ...   
397     200       1  ECF BUYSILVER ECF BUYSILVER ECF BUYWITCH ECF B...   
398     201       0  ECF BUYSILVER ECF BUYSILVER ECF BUYSILVER ECF ...   
399     201       1  ECF BUYSILVER ECF BUYSILVER ECF BUYSILVER ECF ...   

                                                NGrams  
0    [(ECF, ECF), (ECF, ECF), (ECF, BUYWITCH), (BUY...

In [24]:
#function to compute N-gram probabilities, returns a dictionary with the gram as key
#and probability as value. Unobserved grams are contained in the dictionary with a 
#corresponding probability of zero. Inputs are a list of n grams for a given player in 
#a single game and a list of all possible ngrams
def calc_probabilities(ngrams_trace, ngrams_all):
    # Compute the frequency of ngrams in the trace
    frequency_counter = Counter(ngrams_trace)
    
    #calculate frequencies of all ngrams in ngrams_all that appear in the playtrace
    event_count = {gram: frequency_counter.get(gram, 0) for gram in set(ngrams_all)}
    
    #normalise each entry with the number of n-grams observed for that trace, to convert
    #counts into probabilities
    probs = {key: value / (1.0*len(ngrams_trace)) for key, value in event_count.items()}
    
    return probs

#check sum of probabilities is one
example_trace = traces[(traces['GameID'] == 2.0) & (traces['Player'] == 0)]['NGrams'].iloc[0]
calculated_probs = calc_probabilities(example_trace, all_ngrams_list)
total_prob = 0 
for key, value in calculated_probs.items():
    total_prob += value
print(total_prob)

1.0000000000000002


In [ ]:
#next take two traces and compute Jensen-Shannon distance



In [14]:
#calculate frequency distribution
#freq_dist = FreqDist(trace_n_grams)
#for key, value in freq_dist.items():
#    print(f'{key}: {value}')
#print("No of distinct ngrams observed: " + str(len(freq_dist.keys())))

In [ ]:
#check probabilities sum to one